# Sistema de recompensas

In [ ]:
import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import set_random_seed
from sklearn.metrics import accuracy_score

***Read Data***

In [ ]:
data_train = pd.read_csv('../data/interim/bicimad_Xtrain_X_regression_sel.csv', sep=',')
data_test = pd.read_csv('../data/interim/bicimad_Xtest_X_regression_sel.csv', sep=',')

In [ ]:
X_train = data_train.drop(['trip_minutes'], axis=1, inplace=True)
y_train = data_train['trip_minutes']
X_test = data_test.drop(['trip_minutes'], axis=1, inplace=True)
y_test = data_test['trip_minutes']

***Neural Networks***

In [ ]:
set_random_seed(42)

model = Sequential()
model.add(Dense(12, input_shape = (8,), activation = "relu"))
model.add(Dense(8, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model

In [ ]:
model.fit(X_train, y_train, epochs = 50, batch_size = 10)

In [ ]:
_, accuracy = model.evaluate(X_train, y_train)

print(f"Accuracy: {accuracy}")

In [ ]:
y_pred = model.predict(X_test)
y_pred[:15]

In [ ]:
y_pred_round = [round(x[0]) for x in y_pred]
y_pred_round[:15]

In [ ]:
accuracy_score(y_test, y_pred_round)